### CNN's in Pytorch

Code sourced from 

https://github.com/yunjey/pytorch-tutorial/blob/master/tutorials/02-intermediate/convolutional_neural_network/main.py#L35-L56

https://github.com/jcjohnson/pytorch-examples#pytorch-custom-nn-modules

https://cs230-stanford.github.io/pytorch-getting-started.html#tensors-and-variables

In [8]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

In [9]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

like keras, pytorch has some 'pre-bundled' datasets.

see: https://pytorch.org/docs/stable/torchvision/datasets.html


In [10]:
# MNIST dataset
'''

    root (string) – Root directory of dataset where processed/training.pt and processed/test.pt exist.
    train (bool, optional) – If True, creates dataset from training.pt, otherwise from test.pt.
    download (bool, optional) – If true, downloads the dataset from the internet and puts it in root directory. If dataset is already downloaded, it is not downloaded again.
    transform (callable, optional) – A function/transform that takes in an PIL image and returns a transformed version. E.g, transforms.RandomCrop
    target_transform (callable, optional) – A function/transform that takes in the target and transforms it.

'''

train_dataset = torchvision.datasets.MNIST(root='data/',
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='data/',
                                          train=False, 
                                          transform=transforms.ToTensor())



0.00B [00:00, ?B/s]

9.92MB [00:05, 1.67MB/s]                            


Extracting data/MNIST/raw/train-images-idx3-ubyte.gz


0.00B [00:00, ?B/s]

32.8kB [00:00, 36.5kB/s]                            
0.00B [00:00, ?B/s]

Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz


1.65MB [00:02, 616kB/s]                             
0.00B [00:00, ?B/s]

Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz


8.19kB [00:00, 13.9kB/s]                   

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz
Processing...
Done!


In [12]:
# model parameters
num_epochs = 5
num_classes = 10
batch_size = 100
learning_rate = 0.001

In [13]:
# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, shuffle=False)

In [16]:
# Convolutional neural network (two convolutional layers)
class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = nn.Linear(7*7*32, num_classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

In [ ]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x